# Advanced Heuristic

In this project we attempt to analyze the impact of heuristic when solving a problem of searching the winning configuration for the Isolation game.
A heuristic is an approach for finding an approximate solution when classic search methods cannot be used due to various constraints. In Layman's terms it as function to estimate current situation of the game state.

## Heuristics Features

Features of the heuristic function reflect the situation on the board with regards to our goals. I look for 6 kind kinds of different features from the board:

| Feature Code | Feature Name | Description |
| --- | --- | --- |
| F1 | OwnEdgeDistance | In certain stages of the game it is beneficial to stay away from the board edges  |
| F2 | OppEdgeDistance | - |
| F3 | OwnLiberties | Number of liberties available from the state (as described in lectures) |
| F4 | OppLiberties | - |
| F5 | MoveIntoOppOnlyLibery | If oppenent has only one liberty and this liberty is available to us to, we win by moving into it |
| F6 | MoveIntoOppLibery | If we have liberty same as opponent has we move into it thus reducing opponents opportunities |

Obviously the ultimate strategic goal is the only one - to win a game, however the tactical goals maybe different depending on game's advancing stage, thus impact of individual feature might be different. I identified three stages of the game Beginning, Middle and Final and introduced the weight of each feature based on what stage the game at:

| Feature Code | Beginning | Middle | Final |
| --- | --- | --- | --- | 
| F1 | 100 | 10 | 0 |
| F2 | -100 | -10 | 0 |
| F3 | 5 | 20 | 30 |
| F4 | -5 | -10 | -10 |
| F5 | 10000 | 10000 | 10000 | 
| F6 | 0 | 5 | 100 |

As we can see the feature F5 dominates over all features regardless of the stage of the game because it is a definite winning move, thus it has large weight. Feature F6 is not relevant in the beggining of the game, but we exploit every opportunity to move into the cell available for our opponent. Features F1 and F2 tell the agent to stay away from the borad edges in the beggingin of the game (in code I used slightly simplified version that calculates the number of available cells in the radius of 2, sort of density). Features F3 and F4 are changing the style of the game from defensive in the beggining to attacking in the end.
The heuristic of our agent is a summary of all heuristic features with respect to the weights:

$$H=\sum_{i=1}^{6} w_i * F_i$$


## Baseline

The baseline data for our heuristic search are captured using the following environment:

| Parameter | Value | 
| --- | --- |
| Algorithm | Alpha-Beta Prunning |
| Max Depth | 3 |
| Time limit | 300 ms |
| Epochs | 20 |

Result: 

| Algorithm | Win % | Command | 
| --- | --- | --- |
| Greedy | 40% | python run_match.py -f -r 20 -p 4 -o GREEDY |
| Random | 92.5% | python run_match.py -f -r 20 -p 4 -o RANDOM |
| Minimax | 58.8% | python run_match.py -f -r 20 -p 4 -o MINIMAX |
| Self | 50% | python run_match.py -f -r 20 -p 4 -o SELF |



## Analysis

As we can see we have more the 50% of wins against #my_moves - #opponent_moves MINIMAX agent. Surprisingly, our algorithm shows poor performance against GREEDY agent. The reasong might be that in the beginning of the game we rel y heavily on the density of the state e.g. agent attempts to move into the area with as many available moves in the radius of 2. Essentially, it is the same rule as GREEDY agent uses forcing us to make bad decision. After tweaking weight of the parameters F1 and F2 for the beginning phase of the game from (100,-100) to (20,-20) the result looks like:

| Algorithm | Win % | Command | 
| --- | --- | --- |
| Greedy | 85% | python run_match.py -f -r 20 -p 4 -o GREEDY |
| Random | 93.8% | python run_match.py -f -r 20 -p 4 -o RANDOM |
| Minimax | 53.8% | python run_match.py -f -r 20 -p 4 -o MINIMAX |
| Self | 50% | python run_match.py -f -r 20 -p 4 -o SELF |


As we can see below the depth of the search is critical to win a game. With increasing depth from 3 to 5 the winning rate against MINMAX agent grows up to 15%. Unfortunately, the cost is rising exponentially both time wise and memory wise, as we need to explore more nodes. In our test run we had to increase the timeout from 300 to 1500ms to enable the search with depth of 5.

| Algorithm | Win % | Depth | Timeout limit |
| --- | --- | --- | --- |
| Greedy | 85% | 3 | 300 |
| Minimax | 53.8% | 3 | 300 |
| Greedy | 90% | 4 | 1000 |
| Minimax | 72.5% | 4 | 1000 |
| Greedy | 95% | 5 | 1500 |
| Minimax | 75% | 5 | 1500 |